   SpaCy Test
==================
Following tutorial from: https://www.analyticsvidhya.com/blog/2020/06/nlp-project-information-extraction/

In [2]:
# << =========================================================================================== >> 
# <<                                           IMPORTS                                           >> 
# << =========================================================================================== >> 

import spacy
from spacy import displacy
from spacy.matcher import Matcher, DependencyMatcher
from spacy.tokenizer import Tokenizer
# from nltk.tokenize import sent_tokenize
import visualise_spacy_tree
import pandas as pd
from glob import glob
from pathlib import Path
import re
from yaml import safe_load
from typing import List, Callable, Any, Union as Un

pd.set_option("display.max_rows", 128)

In [3]:
# << =========================================================================================== >> 
# <<                                       IMPORT PATTERNS                                       >> 
# << =========================================================================================== >> 
from sys import path

path += ['../model/']

from pat_seller import SELLER_DEP_PATTERNS
from pat_purchaser import PURCHASER_DEP_PATTERNS
from pat_acquired import ACQUIRED_DEP_PATTERNS

ModuleNotFoundError: No module named 'pat_seller'

#### Note:
if you get an error while calling `import_data()` try running the following command:
```py
import nltk
nltk.download('punkt')
```

In [4]:
langmod = spacy.load('en_core_web_sm')
# langmod.add_pipe('sentencizer')
# tokenizer = Tokenizer(langmod.vocab)

In [3]:
# << =========================================================================================== >> 
# <<                                           GLOBALS                                           >> 
# << =========================================================================================== >> 

COL_LABELS = ['content', 'nlp', 'DLRAMT', "ORGS", "ACQUIRED", "SELLER"]
ANS_COL_LABELS = ['ans', "=ACQUIRED", "=ACQBUS", "=ACQLOC", "=DLRAMT", "=PURCHASER", "=SELLER", "=STATUS"]

In [4]:
def GenDict(ids:list, values:list) -> dict:
    gdict:dict = dict()
    for id,val in zip(ids,values):
        gdict.update({id:val})
    return gdict

  Data Handling
-----------------

In [5]:
DOC_IDS = [Path(f).stem for f in glob("../data/development-docs/*")]
DocId = Un[List[str],List[int]]
INPUT_FILES = lambda id: f"../data/development-docs/{id}"
ANS_FILES = lambda id: f"../data/development-anskeys/{id}.key"
GtFl = Callable[[Un[str,int]],str]
# INPUT_FILES = GenDict(DOC_IDS, [f"../data/development-docs/{id}" for id in DOC_IDS])
# ANS_FILES = glob("../data/development-anskeys/*.key")
# ANS_FILES = GenDict(DOC_IDS, [f"../data/development-anskeys/{id}.key" for id in DOC_IDS])
# ANS_KEY_PATTERN = re.compile(r'TEXT: (?P<id>[\d\w]+)\n'
#                              r'AQUIRED: (?P<ACQUIRED>"[^\"]+"|---)\n' +
#                              r'ACQBUS: (?P<ACQBUS>"[^\"]+"|---)\n' +
#                              r'ACQLOC: (?P<ACQLOC>"[^\"]+"|---)\n' +
#                              r'DLRAMT: (?P<DLRAMT>"[^\"]+"|---)\n' +
#                              r'PURCHASER: (?P<PURCHASER>"[^\"]+"|---)\n' +
#                              r'SELLER: (?P<SELLER>"[^\"]+"|---)\n' +
#                              r'STATUS: (?P<STATUS>"[^\"]+"|---)\n'
#                             )
# ANS_KEY_PATTERNS = {'id': re.compile(r'TEXT: ([\d\w]+)'),
#                         'ACQUIRED': re.compile(r'AQUIRED: ("[^\"]+"|---)'),
#                         'ACQBUS': re.compile(r'ACQBUS: ("[^\"]+"|---)'),
#                         'ACQLOC': re.compile(r'ACQLOC: ("[^\"]+"|---)'),
#                         'DLRAMT': re.compile(r'DLRAMT: ("[^\"]+"|---)'),
#                         'PURCHASER': re.compile(r'PURCHASER: ("[^\"]+"|---)'),
#                         'SELLER': re.compile(r'SELLER: ("[^\"]+"|---)'),
#                         'STATUS': re.compile(r'STATUS: ("[^\"]+"|---)')
#                         }
ANS_KEY_LINE_EXTRACT = lambda line: [part.strip().strip('"') for part in line.split(':')]

### Data Cleaning Operations

In [6]:
def clean_text(content:str) -> str:
    """Clean up the sentences & get rid of unessisary whitespace"""
    # normalize line spacing
    content = re.sub(r"\r?\n", ' ', content)
    # remove all extra whitespace
    content = re.sub(r"\s+", ' ', content)
    content = re.sub(r"^\s+", '', content)
    # remove the "Reuter"'s tag at the end of the sentence
    content = re.sub(r"\s*[Rr][Ee][Uu][Tt][Ee][Rr][Ss]?\s*$", '', content)
    # tokenize sentences sentences & split [return]
    return content

In [7]:
def clean_ans(content:str) -> str:
    "Modify contents so that the ans files are valid yaml for easy reading in"
    content = re.sub(r'("[^"]+" */ *"[^"]+")', r"'\1'", content)
    return content

In [8]:
def __getMatch(s:str,id:str) -> str:
    if id in ANS_KEY_PATTERNS:
        match = ANS_KEY_PATTERNS[id].match(s)
        if match is not None and len(match.groups()) > 1:
            return match.group(1)
    return "---"

In [9]:
def import_data(ids:DocId, doc_files:GtFl, ans_files:GtFl=None) -> pd.DataFrame:
    """Produce the dataframes to work with"""
    cols = COL_LABELS if ans_files is None else (COL_LABELS + ANS_COL_LABELS)
    df = pd.DataFrame(index=ids, columns=cols)
    for i in ids:
        with open(doc_files(i), 'r') as docf:
            cont = clean_text(docf.read())
        row = [
            cont,   # 'content'
            langmod(cont), # 'nlp'
            # tokenizer(cont), # 'tokens'
            set(),  # 'ORGS'
            set(),  # 'DLRAMT'
            set(),  # 'ACQUIRED'
            set(),  # 'SELLER'
        ]
        if ans_files is not None:
            with open(ans_files(i),'r') as ansf:
                ans = safe_load(clean_ans(ansf.read()))
            row += [
                ans, # 'ans'
                ans["ACQUIRED"], # "=AQUIRED"
                ans["ACQBUS"],  # "=ACQBUS"
                ans["ACQLOC"], # "=ACQLOC"
                ans["DLRAMT"], # "=DLRAMT"
                ans["PURCHASER"], # "=PURCHASER"
                ans["SELLER"], # "=SELLER"
                ans["STATUS"] # "=STATUS"
            ]
        df.loc[i] = row
    return df

# def import_data(ids:DocId, doc_files:GtFl, ans_files:GtFl=None) -> pd.DataFrame:
#     """Produce the dataframes to work with"""
#     cols = COL_LABELS if ans_files is None else (COL_LABELS + ANS_COL_LABELS)
#     df = pd.DataFrame(index=ids, columns=cols)
#     for i in ids:
#         # df.loc[i,'id'] = i 
#         with open(doc_files(i), 'r') as docf:
#             cont = clean_text(docf.read())
#         df.loc[i,'content'] = cont
#         print(f"{i} df.loc[i,'nlp'] : {df.loc[i,'nlp']}")
#         df.loc[i,'nlp'] = langmod(cont)
#         print(f"        -> {df.loc[i,'nlp'][:12]}")
#         df.loc[i,'tokens'] = tokenizer(cont)
#         df.loc[i,'ORGS'] = set()
#         df.loc[i,'DLRAMT'] = set()
#         if ans_files is not None:
#             with open(ans_files(i),'r') as ansf:
#                 ans = safe_load(ansf)
#             # df.loc[i,'ans'] = ans
#             for k in ans:
#                 if k == 'TEXT':
#                     continue
#                 df.loc[i,'='+k] = ans[k]
#     return df

### Get the data

In [10]:
docs = import_data(DOC_IDS[:16]+['1099'], INPUT_FILES, ANS_FILES)
docs

D:\dev-tools\python\Anaconda\envs\secondary-base\lib\site-packages\pandas\core\internals\blocks.py:983: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


content  \
10042  Meridian Energy Inc and Castone Development Co...   
10066  Security Pacific Corp and Users Inc, a credit ...   
10158  Americanture Inc said it has purchased America...   
10171  Viacom International Inc said it set April 6 a...   
1029   Scott's Hospitality Inc said it acquired all i...   
1032   Video Display Corfp said it has reached a tent...   
10348  Applied chemicals group Henkel KGaA said it is...   
10371  Union Miniere SA said in a statement that it h...   
10473  Honeywell Inc said it has completed the sale o...   
10527  LSB Industries Inc said it entered into an agr...   
10541  Plaza Group said it completed a transaction in...   
10674  BTR Nylex Ltd said it will increase its takeov...   
10694  Woolworth Holdings Plc said it would make a 24...   
10714  Tesco Plc said County Bank Ltd had bought 165,...   
10721  Sweden's Wallenberg group said it raised its h...   
10787  Suffield Financial Corp said it has received a...   
1099   Norton co said it has agreed to sell its remai...   

                                                     nlp DLRAMT ORGS ACQUIRED  \
10042  (Meridian, Energy, Inc, and, Castone, Developm...     {}   {}       {}   
10066  (Security, Pacific, Corp, and, Users, Inc, ,, ...     {}   {}       {}   
10158  (Americanture, Inc, said, it, has, purchased, ...     {}   {}       {}   
10171  (Viacom, International, Inc, said, it, set, Ap...     {}   {}       {}   
1029   (Scott, 's, Hospitality, Inc, said, it, acquir...     {}   {}       {}   
1032   (Video, Display, Corfp, said, it, has, reached...     {}   {}       {}   
10348  (Applied, chemicals, group, Henkel, KGaA, said...     {}   {}       {}   
10371  (Union, Miniere, SA, said, in, a, statement, t...     {}   {}       {}   
10473  (Honeywell, Inc, said, it, has, completed, the...     {}   {}       {}   
10527  (LSB, Industries, Inc, said, it, entered, into...     {}   {}       {}   
10541  (Plaza, Group, said, it, completed, a, transac...     {}   {}       {}   
10674  (BTR, Nylex, Ltd, said, it, will, increase, it...     {}   {}       {}   
10694  (Woolworth, Holdings, Plc, said, it, would, ma...     {}   {}       {}   
10714  (Tesco, Plc, said, County, Bank, Ltd, had, bou...     {}   {}       {}   
10721  (Sweden, 's, Wallenberg, group, said, it, rais...     {}   {}       {}   
10787  (Suffield, Financial, Corp, said, it, has, rec...     {}   {}       {}   
1099   (Norton, co, said, it, has, agreed, to, sell, ...     {}   {}       {}   

      SELLER                                                ans  \
10042     {}  {'TEXT': 10042, 'ACQUIRED': 'Castone Developme...   
10066     {}  {'TEXT': 10066, 'ACQUIRED': 'Users Inc', 'ACQB...   
10158     {}  {'TEXT': 10158, 'ACQUIRED': 'American Adventur...   
10171     {}  {'TEXT': 10171, 'ACQUIRED': 'Arsenal Acquiring...   
1029      {}  {'TEXT': 1029, 'ACQUIRED': 'Capital Food Servi...   
1032      {}  {'TEXT': 1032, 'ACQUIRED': 'cable television b...   
10348     {}  {'TEXT': 10348, 'ACQUIRED': 'Noblee und Thoerl...   
10371     {}  {'TEXT': 10371, 'ACQUIRED': 'Pancontinental Mi...   
10473     {}  {'TEXT': 10473, 'ACQUIRED': 'Honeywell Informa...   
10527     {}  {'TEXT': 10527, 'ACQUIRED': 'Northwest Federal...   
10541     {}  {'TEXT': 10541, 'ACQUIRED': 'Flyfaire Internat...   
10674     {}  {'TEXT': 10674, 'ACQUIRED': 'Borg-Warner (Aust...   
10694     {}  {'TEXT': 10694, 'ACQUIRED': 'Superdrug Stores ...   
10714     {}  {'TEXT': 10714, 'ACQUIRED': 'Hillards Plc', 'A...   
10721     {}  {'TEXT': 10721, 'ACQUIRED': 'Telefon AB L.M. E...   
10787     {}  {'TEXT': 10787, 'ACQUIRED': 'Coastal Bancorp',...   
1099      {}  {'TEXT': 1099, 'ACQUIRED': 'Norton co Pty Ltd'...   

                                            =ACQUIRED  \
10042                        Castone Development Corp   
10066                                       Users Inc   
10158                          American Adventure Inc   
10171                          Arsenal Acquiring Corp   
1029             

## Sale Price Amount Experimental Extraction (SpaCy)

### Manually created SpaCy Pattern

In [11]:
PATS_SALE_PRICE = [
    [ # ...
        {'IS_DIGIT': True, 'LIKE_NUM': True, 'OP':'+'}, # numerical values
        {'LIKE_NUM': True, 'OP':'?', 'IS_ALPHA': True}, #{'IN': ['mln', 'bln']}}, # numerical values like million
        {'POS':'ADJ', 'OP':'?'}, # curency regionally identifiers
        {'POS': 'NOUN'}, # {'IN': ['dlrs', 'p', 'pence', 'pound', 'pounds', 'stg', 'yen']}}, # curency type identifiers
        {'LOWER': "cash", 'OP':'?'}
    ],
]

In [12]:
# test the basics
def dlrs_test(text):
    res = list()
    doc_nlp = langmod(text)
    matcher = Matcher(langmod.vocab)
    matcher.add('DLRAMT', PATS_SALE_PRICE)
    matches = matcher(doc_nlp)
    # finding patterns in the text
    for i in range(0,len(matches)):
        # match: id, start, end
        token = doc_nlp[matches[i][1]:matches[i][2]]
        # append token to list
        res.append(str(token))
    return res if len(res) > 0 else "---"

# docs['DLRAMNT'] = docs['content'].apply(dlrs_test)
# docs

In [13]:
def ner_test(df:pd.DataFrame) -> None:
    for i,row in df.iterrows():
        for ent in row['nlp'].ents:
            if ent.label_ == "ORG":
                row['ORGS'] += ent.text
            elif ent.label_ == "MONEY": 
                row['DLRAMT'] += ent.text
    
# ner_test(docs)
# docs['ORGS']

In [14]:
displacy.render(docs.loc['1099','nlp'], style='ent')

In [15]:
displacy.render(docs.loc['1099','nlp'], style='dep')

In [33]:
for id in docs.index:
    nlp = docs.loc[id,'nlp']
    cont = docs.loc[id,'content']
    depmatch = DependencyMatcher(nlp.vocab)
    # for pat in SELLER_DEP_PATTERNS:
    #     print(pat)
    depmatch.add("SELLER", SELLER_DEP_PATTERNS)
    depmatch.add("ACQUIRED", ACQUIRED_DEP_PATTERNS)
    seller = set()
    acq = set()
    matches = depmatch(nlp)
    s = 0
    a = 0
    for pat,toks in matches:
        print(pat)
        if pat == "SELLER":
            seller |= {nlp[toks[i]].text for i in range(len(toks)) if "SELLER" in SELLER_DEP_PATTERNS[s][i]['RIGHT_ID']} # filter by 'RIGHT_ID' then get ent_id
            s += 1
        elif pat == "ACQUIRED":
            acq |= {nlp[toks[i]].text for i in range(len(toks)) if "ACQUIRED" in ACQUIRED_DEP_PATTERNS[a][i]['RIGHT_ID']} # filter by 'RIGHT_ID' then get ent_id
            a += 1
    docs.loc[id,'SELLER'] = seller
    docs.loc[id,'ACQUIRED'] = acq
docs

18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
18118150269197602181
13008773625823506268


content  \
10042  Meridian Energy Inc and Castone Development Co...   
10066  Security Pacific Corp and Users Inc, a credit ...   
10158  Americanture Inc said it has purchased America...   
10171  Viacom International Inc said it set April 6 a...   
1029   Scott's Hospitality Inc said it acquired all i...   
1032   Video Display Corfp said it has reached a tent...   
10348  Applied chemicals group Henkel KGaA said it is...   
10371  Union Miniere SA said in a statement that it h...   
10473  Honeywell Inc said it has completed the sale o...   
10527  LSB Industries Inc said it entered into an agr...   
10541  Plaza Group said it completed a transaction in...   
10674  BTR Nylex Ltd said it will increase its takeov...   
10694  Woolworth Holdings Plc said it would make a 24...   
10714  Tesco Plc said County Bank Ltd had bought 165,...   
10721  Sweden's Wallenberg group said it raised its h...   
10787  Suffield Financial Corp said it has received a...   
1099   Norton co said it has agreed to sell its remai...   

                                                     nlp DLRAMT ORGS ACQUIRED  \
10042  (Meridian, Energy, Inc, and, Castone, Developm...     {}   {}       {}   
10066  (Security, Pacific, Corp, and, Users, Inc, ,, ...     {}   {}       {}   
10158  (Americanture, Inc, said, it, has, purchased, ...     {}   {}       {}   
10171  (Viacom, International, Inc, said, it, set, Ap...     {}   {}       {}   
1029   (Scott, 's, Hospitality, Inc, said, it, acquir...     {}   {}       {}   
1032   (Video, Display, Corfp, said, it, has, reached...     {}   {}       {}   
10348  (Applied, chemicals, group, Henkel, KGaA, said...     {}   {}       {}   
10371  (Union, Miniere, SA, said, in, a, statement, t...     {}   {}       {}   
10473  (Honeywell, Inc, said, it, has, completed, the...     {}   {}       {}   
10527  (LSB, Industries, Inc, said, it, entered, into...     {}   {}       {}   
10541  (Plaza, Group, said, it, completed, a, transac...     {}   {}       {}   
10674  (BTR, Nylex, Ltd, said, it, will, increase, it...     {}   {}       {}   
10694  (Woolworth, Holdings, Plc, said, it, would, ma...     {}   {}       {}   
10714  (Tesco, Plc, said, County, Bank, Ltd, had, bou...     {}   {}       {}   
10721  (Sweden, 's, Wallenberg, group, said, it, rais...     {}   {}       {}   
10787  (Suffield, Financial, Corp, said, it, has, rec...     {}   {}       {}   
1099   (Norton, co, said, it, has, agreed, to, sell, ...     {}   {}       {}   

      SELLER                                                ans  \
10042     {}  {'TEXT': 10042, 'ACQUIRED': 'Castone Developme...   
10066     {}  {'TEXT': 10066, 'ACQUIRED': 'Users Inc', 'ACQB...   
10158     {}  {'TEXT': 10158, 'ACQUIRED': 'American Adventur...   
10171     {}  {'TEXT': 10171, 'ACQUIRED': 'Arsenal Acquiring...   
1029      {}  {'TEXT': 1029, 'ACQUIRED': 'Capital Food Servi...   
1032      {}  {'TEXT': 1032, 'ACQUIRED': 'cable television b...   
10348     {}  {'TEXT': 10348, 'ACQUIRED': 'Noblee und Thoerl...   
10371     {}  {'TEXT': 10371, 'ACQUIRED': 'Pancontinental Mi...   
10473     {}  {'TEXT': 10473, 'ACQUIRED': 'Honeywell Informa...   
10527     {}  {'TEXT': 10527, 'ACQUIRED': 'Northwest Federal...   
10541     {}  {'TEXT': 10541, 'ACQUIRED': 'Flyfaire Internat...   
10674     {}  {'TEXT': 10674, 'ACQUIRED': 'Borg-Warner (Aust...   
10694     {}  {'TEXT': 10694, 'ACQUIRED': 'Superdrug Stores ...   
10714     {}  {'TEXT': 10714, 'ACQUIRED': 'Hillards Plc', 'A...   
10721     {}  {'TEXT': 10721, 'ACQUIRED': 'Telefon AB L.M. E...   
10787     {}  {'TEXT': 10787, 'ACQUIRED': 'Coastal Bancorp',...   
1099      {}  {'TEXT': 1099, 'ACQUIRED': 'Norton co Pty Ltd'...   

                                            =ACQUIRED  \
10042                        Castone Development Corp   
10066                                       Users Inc   
10158                          American Adventure Inc   
10171                          Arsenal Acquiring Corp   
1029             

In [29]:
docs.loc['1099',['ACQUIRED','SELLER']]

ACQUIRED                                                   {}
SELLER      [(None, [[{'RIGHT_ID': "VP('sell')", 'RIGHT_AT...
Name: 1099, dtype: object

## Tokenizer Tests 
> (not necessary to use a tokenizer, tokens accessible via for lop on nlp)

In [20]:
# docs.loc['1099','tokens'][0].pos

In [21]:
docs.loc['1099','nlp']

Norton co said it has agreed to sell its remaining South African business, Norton co Pty Ltd, to Global Mining and Industrial Corp of South africa for undisclosed terms. The company said the unit accounts for less than two pct of Norton revenues and is being sold because "Growing societal pressures in the United States and the unsettled situation in south Africa had required a disproportionate amount of management tiome to oversee." Norton said the unit is profitable. Norton said it will provide the South Afrcian unit with future technical support, and products makde under that agreement will continue to be marketed under the Norton trademark.

In [22]:
docs.loc['1099','nlp'][0].pos

96

In [40]:
print("TAGGER id's (POS and the like):")
for thing in ['$', "''", ',', '-LRB-', '-RRB-', '.', ':', "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "``"]:
    print(f"{thing:>6} :: {spacy.explain(thing)}")

TAGGER id's (POS and the like):
     $ :: symbol, currency
    '' :: closing quotation mark
     , :: punctuation mark, comma
 -LRB- :: left round bracket
 -RRB- :: right round bracket
     . :: punctuation mark, sentence closer
     : :: punctuation mark, colon or ellipsis
   ADD :: email
   AFX :: affix
    CC :: conjunction, coordinating
    CD :: cardinal number
    DT :: determiner
    EX :: existential there
    FW :: foreign word
  HYPH :: punctuation mark, hyphen
    IN :: conjunction, subordinating or preposition
    JJ :: adjective (English), other noun-modifier (Chinese)
   JJR :: adjective, comparative
   JJS :: adjective, superlative
    LS :: list item marker
    MD :: verb, modal auxiliary
   NFP :: superfluous punctuation
    NN :: noun, singular or mass
   NNP :: noun, proper singular
  NNPS :: noun, proper plural
   NNS :: noun, plural
   PDT :: predeterminer
   POS :: possessive ending
   PRP :: pronoun, personal
  PRP$ :: pronoun, possessive
    RB :: adverb
   RBR 

In [41]:
print("PARSER id's (dependency tree stuff):")
for thing in "ROOT, acl, acomp, advcl, advmod, agent, amod, appos, attr, aux, auxpass, case, cc, ccomp, compound, conj, csubj, csubjpass, dative, dep, det, dobj, expl, intj, mark, meta, neg, nmod, npadvmod, nsubj, nsubjpass, nummod, oprd, parataxis, pcomp, pobj, poss, preconj, predet, prep, prt, punct, quantmod, relcl, xcomp".split(', '):
    print(f"{thing:>10} :: {spacy.explain(thing)}")

PARSER id's (dependency tree stuff):
      ROOT :: None
       acl :: clausal modifier of noun (adjectival clause)
     acomp :: adjectival complement
     advcl :: adverbial clause modifier
    advmod :: adverbial modifier
     agent :: agent
      amod :: adjectival modifier
     appos :: appositional modifier
      attr :: attribute
       aux :: auxiliary
   auxpass :: auxiliary (passive)
      case :: case marking
        cc :: coordinating conjunction
     ccomp :: clausal complement
  compound :: compound
      conj :: conjunct
     csubj :: clausal subject
 csubjpass :: clausal subject (passive)
    dative :: dative
       dep :: unclassified dependent
       det :: determiner
      dobj :: direct object
      expl :: expletive
      intj :: interjection
      mark :: marker
      meta :: meta modifier
       neg :: negation modifier
      nmod :: modifier of nominal
  npadvmod :: noun phrase as adverbial modifier
     nsubj :: nominal subject
 nsubjpass :: nominal subject (pas

In [43]:
print("NER id's (kind of noun):")
for thing in "CARDINAL, DATE, EVENT, FAC, GPE, LANGUAGE, LAW, LOC, MONEY, NORP, ORDINAL, ORG, PERCENT, PERSON, PRODUCT, QUANTITY, TIME, WORK_OF_ART".split(', '):
    print(f"{thing:>10} :: {spacy.explain(thing)}")

NER id's (kind of noun):
  CARDINAL :: Numerals that do not fall under another type
      DATE :: Absolute or relative dates or periods
     EVENT :: Named hurricanes, battles, wars, sports events, etc.
       FAC :: Buildings, airports, highways, bridges, etc.
       GPE :: Countries, cities, states
  LANGUAGE :: Any named language
       LAW :: Named documents made into laws.
       LOC :: Non-GPE locations, mountain ranges, bodies of water
     MONEY :: Monetary values, including unit
      NORP :: Nationalities or religious or political groups
   ORDINAL :: "first", "second", etc.
       ORG :: Companies, agencies, institutions, etc.
   PERCENT :: Percentage, including "%"
    PERSON :: People, including fictional
   PRODUCT :: Objects, vehicles, foods, etc. (not services)
  QUANTITY :: Measurements, as of weight or distance
      TIME :: Times smaller than a day
WORK_OF_ART :: Titles of books, songs, etc.


In [45]:
print(str(docs.loc['1099','nlp'][1:12]))

co said it has agreed to sell its remaining South African


In [7]:
displacy.render(langmod(clean_text("""
                                   First Wisconsin Corp said it
plans to acquire Shelard Bancshares Inc for about 25 mln dlrs
in cash, its first acquisition of a Minnesota-based bank.
    First Wisconsin said Shelard is the holding company for two
banks with total assets of 168 mln dlrs.
    First Wisconsin, which had assets at yearend of 7.1 billion
dlrs, said the Shelard purchase price is about 12 times the
1986 earnings of the bank.
    It said the two Shelard banks have a total of five offices
in the Minneapolis-St. Paul area.
                        """)), style='dep')

In [8]:
displacy.render(langmod(clean_text("""
                                   Holding company Gemina Spa said it had
no comment on an Italian press report today that it has
acquired a 12 pct stake in Nuovo Banco Ambrosiano Spa.
    Italian daily La Repubblica said that Gemina paid 205
billion lire for the shareholding from several Italian banks. A
Gemina spokeswoman told Reuters, "We have nothing to say about
the report."
    A spokeswoman for Milan-based Ambrosiano said, "We have no
information at this time." In February, Ambrosiano chairman
Giovanni Bazoli said foreign and domestic firms have expressed
interest in buying stakes in the bank.
    REUTER
                        """)), style='dep')